In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.5 MB/s eta 0:00:00


In [ ]:

!pip3 install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
#from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification 
from torch.nn.utils.rnn import pad_sequence

# Data loading 
df = pd.read_csv('clean_df_top_clusters_10id.csv')

print(df['cluster_id'].unique())

[ 2  3  8  9 16 22 60 64 78 80]


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)

In [ ]:
# Data loading 
from tqdm import tqdm
df = pd.read_csv('clean_df_top_clusters_10id.csv')

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Creating a dictionary for reinterpreting cluster_id values
cluster_mapping = {2: 0, 3: 1, 8: 2, 9: 3, 16: 4, 22: 5, 60: 6, 64: 7, 78: 8, 80: 9}

# Reassignments train_df, val_df и test_df
train_df['cluster_id'] = train_df['cluster_id'].map(cluster_mapping)
val_df['cluster_id'] = val_df['cluster_id'].map(cluster_mapping)
test_df['cluster_id'] = test_df['cluster_id'].map(cluster_mapping)

# Loading the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocessing of data
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)

# Converting data to PyTorch format
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_df['cluster_id'].tolist())
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_df['cluster_id'].tolist())
)
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_df['cluster_id'].tolist())
)

# Loading a pre-trained BERT model for sequence classification
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(df['cluster_id'].unique()))

# Installing the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#Creating a DataLoader for training, validation and testing
batch_size = 32   
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialization of the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)

# Model Training
model.train()
for epoch in range(3):
    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    for batch in val_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            val_loss += loss_fn(outputs.logits, labels).item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss = {loss.item()}, Val Loss = {val_loss}")

# Saving the model
model.save_pretrained(".")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1: Train Loss = 1.8488736152648926, Val Loss = 2.1104430395203666


100%|██████████| 2362/2362 [3:48:33<00:00,  5.81s/it]  


Epoch 2: Train Loss = 2.2524468898773193, Val Loss = 2.0990756037267477


100%|██████████| 2362/2362 [3:58:51<00:00,  6.07s/it]  


Epoch 3: Train Loss = 1.8473656177520752, Val Loss = 2.3611829969528557


In [ ]:
# Testing
model.eval()
predictions = []
for batch in test_loader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_classes = logits.argmax(dim=1).tolist()
        predictions.extend(predicted_classes)

print(predictions)


[1, 2, 0, 2, 2, 0, 8, 2, 0, 3, 5, 0, 0, 3, 3, 1, 0, 2, 2, 2, 1, 7, 5, 5, 0, 0, 7, 2, 0, 3, 2, 5, 3, 0, 0, 7, 0, 0, 3, 3, 6, 2, 0, 3, 2, 9, 0, 5, 9, 2, 2, 5, 2, 2, 3, 0, 1, 2, 2, 2, 8, 8, 3, 2, 3, 0, 0, 0, 3, 1, 2, 2, 0, 8, 2, 0, 3, 0, 0, 2, 0, 0, 2, 0, 2, 6, 4, 5, 0, 2, 2, 2, 3, 0, 1, 1, 0, 0, 2, 1, 5, 2, 0, 2, 3, 2, 0, 3, 0, 3, 0, 2, 3, 2, 2, 8, 0, 0, 3, 1, 2, 2, 0, 3, 2, 2, 0, 8, 9, 5, 2, 1, 2, 0, 3, 2, 2, 2, 2, 0, 5, 0, 0, 0, 8, 8, 2, 0, 2, 5, 3, 5, 8, 0, 2, 0, 0, 2, 0, 2, 0, 7, 2, 1, 3, 2, 8, 2, 0, 5, 2, 0, 7, 0, 8, 0, 1, 0, 7, 8, 0, 9, 3, 3, 8, 0, 8, 0, 8, 3, 0, 2, 1, 3, 3, 3, 0, 0, 3, 0, 0, 1, 2, 0, 0, 8, 8, 3, 2, 2, 3, 2, 3, 2, 4, 2, 8, 5, 3, 2, 8, 5, 0, 0, 0, 2, 2, 2, 0, 2, 2, 7, 8, 2, 3, 8, 8, 0, 2, 3, 0, 0, 3, 2, 0, 0, 0, 3, 3, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 8, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 1, 5, 2, 2, 3, 2, 3, 2, 8, 2, 0, 0, 0, 3, 2, 0, 0, 0, 2, 5, 2, 2, 1, 0, 0, 2, 3, 2, 1, 3, 0, 5, 7, 0, 2, 8, 2, 0, 7, 1, 0, 3, 2, 0, 2, 0, 8, 3, 3, 2, 2, 0, 0, 3, 0, 2, 2, 6, 0, 2, 2, 3, 2, 